In [ ]:
from HARP.scan_anal import Scan
from HARP.iris_functions import Iris
from HARP.image_processing import Image
import os
import numpy as np 
import matplotlib.pyplot as plt
from functions import Adam_calib
import pandas as pd
import json



In [ ]:

# Define where to save the processed scan
save_folder = '/Volumes/KasiaDrive/Processed_Artemis'


#Where to take the scan data from
data_folder = '/Volumes/KasiaDrive/Data/Artemis'
#data_folder =os.path.join(data_folder,'AndorScan-021-20220624-1310 - MgO 200mm Rotation Intensity')
h5files = [i for i in os.listdir(data_folder) if i != 'Camera.h5' and 'h5' in i]


In [ ]:
os.listdir(data_folder)
h5files = ['ImgData-000-20220629-210037.h5']

In [ ]:
h5files

In [ ]:

#Open the Scan class
scan = Scan()
scan.set_folder(data_folder)
# fill out which are available from 
scan.set_params(iris=-1, wedge=None, rotation=0, MCPPos=30000) # here fill out the variables that you are not scanning through. This scan is Rotation and iris so I fill out the other MCPPos and Wedge
scan.set_verlim(0, -1) #this is the vertical cropping you want to do
scan.set_eVlim((0,40)) #this is the energy limits you want to look at
scan.wl0 = 780e-9

#To add all files or at least multiples uncomment:
# for f in h5files[:3]:
#    try:
#        scan.populate_scan_h5(f, function=Adam_calib)
#    except:
#        print('Failed to add {}'.format(f))


# for h5 in h5files:
#     try:
#         iris = h5.split('_')[1][:2]
#         scan.populate_scan_h5(h5, function=Adam_calib)
#         mask = scan.scan_data['iris']==-1
#         scan.scan_data.loc[mask, 'iris'] = int(iris)
#     except:
#         print('unable to open h5', h5)


In [ ]:

#scan.populate_scan_h5(h5files[0], function=Adam_calib)
scan.populate_scan_h5(h5files[0], function=Adam_calib)

In [ ]:
#
date = '220621'
sample = 'MgO',
wl0 = '800'
set_params = 'part2_polar_rotation_comparison_course'
MCPpos = 30000

save_name = 'Scan_{}_{}um_{}nm_{}_{}MCP.npz'.format(date, sample, wl0, set_params, MCPpos)



In [ ]:
scan.scan_data

In [ ]:
scan.scan_data['waveplate'] = scan.scan_data['waveplate 1']
scan.scan_data['wp2'] = scan.scan_data['waveplate 2']
scan.scan_data['rotation'] = scan.scan_data['sample rotation']
scan.scan_data

In [ ]:
test1 = scan.scan_data.copy()
test1['data'] = test1.apply(lambda row: row.Data.data, axis=1)
test1['eV'] = test1.apply(lambda row: row.Data.eV_axis, axis=1)
test1['ver'] = test1.apply(lambda row: row.Data.ver_lim, axis=1)
test1['wl0'] = test1.apply(lambda row: row.Data.wl0, axis=1)
test1 = test1.drop(['Data'], axis=1)
np.savez(os.path.join(save_folder, save_name), 
            iris = test1['iris'].values, 
            rotation=test1['rotation'].values,
            mcppos=test1['MCP Pos'].values, 
            data=test1['data'].values,
            ver=test1['ver'].values,
            wl0=test1['wl0'].values,
            eV = test1['eV'].values, 
            wp = test1['waveplate'].values,
            wp2 = test1['wp2'].values
        )